In [ ]:

from numpy import *
set_printoptions(legacy = '1.25')

from scipy.stats import t as T, norm as Z
from matplotlib.pyplot import *

t = arange(-3, 3, .01)
for d in [3,4,7]: 
	plot(t, T(d).pdf(t), label = f'd = {d}')

plot(t, Z.pdf(t), '--', label = r'd = $\infty$')
grid()
legend()
show()   


In [ ]:

# p = P(T(d) < tscore)
def t_pvalue(d, t): return T(d).cdf(t)

# alpha = P(T(d) > tscore)	
def t_score(d, alpha): return T(d).ppf(1 - alpha)


In [ ]:

##########################
# Margin of Error - T
##########################
	
def margin_of_error(sample, alpha, tail):
	n = len(sample)
	s = sqrt(cov(sample, bias = False))
	d = n - 1
	if tail == 'two-tail': 
		t = t_score(d, alpha/2)
	elif tail == 'lower-tail' or tail == 'upper-tail': 
		t = t_score(d, alpha)
	else: return 'What is the tail?'
	epsilon = s * t / sqrt(n)
	return epsilon


In [ ]:

sample = arange(118,128)
alpha = .01
tail = 'lower-tail'
epsilon = margin_of_error(sample, alpha, tail)

xbar = mean(sample)

epsilon, xbar


In [ ]:

##########################
# Confidence Interval - T
##########################

def confidence_interval(sample, alpha, tail):
	xbar = mean(sample)
	epsilon = margin_of_error(sample, alpha, tail)
	if tail == 'two-tail': 
		U = xbar + epsilon
		L = xbar - epsilon
	elif tail == 'upper-tail': 
		U = xbar + epsilon
		L = -inf
	elif tail == 'lower-tail': 
		L = xbar - epsilon
		U = inf
	else: return 'What is the tail?'
	return L, U
	
L, U = confidence_interval(sample, alpha, tail)
print(f'lower, upper: {L}, {U}')


In [ ]:

###################
# Hypothesis T-test
###################

def t_test(mu0, sample, alpha, tail):
	L, U = confidence_interval(sample, alpha, tail)
	if tail == 'lower-tail': 
		if mu0 < L: return 'reject H0'
		else: return 'do not reject H0'
	elif tail == 'upper-tail':
		if mu0 > U: return 'reject H0'
		else: return 'do not reject H0'
	elif tail == 'two-tail':
		if mu0 < L or mu0 > U: return 'reject H0'
		else: return 'do not reject H0'
	else: return 'What is the tail?'
	
mu0 = 120

decision = t_test(mu0, sample, alpha, tail)
print(f'decision: {decision}')


In [ ]:

############################
# Type1 and Type2 errors - T
############################
	
def rejection_power(mu0, mu, sample, alpha, tail):
	s = sqrt(cov(sample, bias = False))
	n = len(sample)
	delta = sqrt(n) * (mu0 -  mu) / s
	d = n - 1
	if tail == 'lower-tail': 
		t = t_score(d, alpha)
		beta = t_pvalue(d, delta + t)
	elif tail == 'upper-tail': 
		t = t_score(d, alpha)
		beta = 1 - t_pvalue(d, delta - t)
	elif tail == 'two-tail': 
		t = t_score(d, alpha/2)
		beta = t_pvalue(d, delta + t) - t_pvalue(d, delta - t)
	else: return 'What is the tail?'
	power = 1 - beta
	return power
	
mu = 122
	
power = rejection_power(mu0, mu, sample, alpha, tail)
print(f'power: {power}')
